In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import pandas as pd
import torch
from torchmetrics.functional.text import word_error_rate
from torchmetrics.functional.text import char_error_rate

from src.models.carbune_module import CarbuneLitModule2
from src.models.carbune_module import LitModule1
from src.models.components.carbune2020_net import Carbune2020NetAttempt1
from src.utils.io import load_alphabet
from src.data.tokenisers import AlphabetMapper
from src.data.online_handwriting_datamodule import IAMOnDBDataModule

In [3]:
PATH = '../logs/train/multiruns/2024-03-20_13-11-14/0/tensorboard/version_0/checkpoints/epoch=9999-step=1900000.ckpt'
PATH = '../logs/train/multiruns/2024-03-27_14-06-58/0/checkpoints/epoch_epoch=029.ckpt'

BASE_PATH = Path('../logs/train/multiruns/2024-03-28_11-36-50/0')
CHECKPOINT_PATH = BASE_PATH / 'checkpoints/epoch000999.ckpt'

It looks like I don't have models saved :-D.

Next steps:

1. [x] Add model checkpointing w/ new config.
2. [x] RUn a short training just to get some model to play around with.
3. Use some of the following links to load model for inference. Links:
    - [1](https://lightning.ai/docs/pytorch/stable/common/lightning_module.html#save-hyperparameters), [2](https://lightning.ai/docs/pytorch/stable/common/trainer.html#inference-mode), [3](https://lightning.ai/docs/pytorch/stable/deploy/production_basic.html), [4](https://lightning.ai/docs/pytorch/stable/common/checkpointing_basic.html#nn-module-from-checkpoint).
    - G"pytorch lightning load model for inference"
    - !! https://lightning.ai/docs/pytorch/stable/common/lightning_module.html#inference-in-production or
    - https://lightning.ai/docs/pytorch/stable/common/lightning_module.html#save-hyperparameters
    - https://lightning.ai/forums/t/save-load-model-for-inference/542
    - https://pytorch.org/tutorials/recipes/recipes/saving_and_loading_a_general_checkpoint.html
4. Then load IAMonDB dataset and do inference. Consider using `Trainer` instead of DIY approach of doing inference. That's b/c it's easier and will not be seen in the promo video anyways.
4. Then run full training to get `checkpoint` data from there to play around.
4. Then load handwritten X++pagewise dataset and do inference on them
4. Then look for method to draw stroke in Jupyter Notebook.

*Note:* Make sure that I do correct pre-processing!

# Try to complete point 3

## Attempt 1

In [4]:
BASE_PATH = Path('../logs/train/multiruns/2024-04-04_23-07-31/0')
CHECKPOINT_PATH = BASE_PATH / 'checkpoints/epoch000549.ckpt'

BASE_PATH = Path('../logs/train/multiruns/2024-04-06_10-29-48/0')
CHECKPOINT_PATH = BASE_PATH / 'checkpoints/epoch000099.ckpt'

BASE_PATH = Path('../logs/train/multiruns/2024-04-07_13-03-01/0')
CHECKPOINT_PATH = BASE_PATH / 'checkpoints/epoch000649.ckpt'

model = LitModule1.load_from_checkpoint(CHECKPOINT_PATH)

/localdisk/s1691089/venvs/carbune2020/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'decoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['decoder'])`.


In [6]:
model.eval()

LitModule1(
  (criterion): CTCLoss()
  (log_softmax): LogSoftmax(dim=2)
  (lstm_stack): LSTM(3, 64, num_layers=3, bidirectional=True)
  (linear): Linear(in_features=128, out_features=82, bias=True)
)

In [7]:
# TODO:
# - Remove other attemtps to open checkpoint
# - Check if I am good to ignore CTCDecoder indeed
# - tidy up code here
# - try to do inference w/ 'iam_SimpleNormalise_xyn' model!

# - OTHER: THINK ABOUT BEAM DECODER!

In [7]:
checkpoint = torch.load(CHECKPOINT_PATH, map_location=lambda storage, loc: storage)


In [8]:
list(checkpoint.keys())

['epoch',
 'global_step',
 'pytorch-lightning_version',
 'state_dict',
 'loops',
 'callbacks',
 'optimizer_states',
 'lr_schedulers',
 'hparams_name',
 'hyper_parameters',
 'datamodule_hparams_name',
 'datamodule_hyper_parameters']

In [9]:
alphabet = load_alphabet(BASE_PATH / 'alphabet.json')

In [10]:
alphabet_mapper = AlphabetMapper( alphabet )

In [11]:
decoder = checkpoint['hyper_parameters']['decoder']

### Load data: From IAMOnDB

In [21]:
dm = IAMOnDBDataModule(
    '../data/datasets/IAM-OnDB',
    batch_size=64,
    train_val_test_split=[0.8, 0.2, 0],
    num_workers=4,
    pin_memory=True,
    # limit=200,
    limit=-1,
    transform='iam_SimpleNormalise_xyn',
)
dm.setup()
dl_val = dm.val_dataloader()
dl_train = dm.train_dataloader()

/localdisk/s1691089/venvs/carbune2020/lib/python3.10/site-packages/torch/utils/data/dataset.py:414: UserWarning: Length of split at index 2 is 0. This might result in an empty dataset.
  warnings.warn(f"Length of split at index {i} is 0. "


### Do inference: IAMOnDB

In [30]:
df_data = {
    'name': [],
    'type': [],
    'value': [],
}

for name, dloader in {'train': dl_train, 'val': dl_val }.items():
    for sample_batched in dloader:
        batch = sample_batched # Just a shortcut

        log_softmax = model(sample_batched['ink'].to('cuda'))

        decoded_texts = decoder(log_softmax, alphabet_mapper)

        # TODO: Put this in function to use both here and in model training -> or just use underlying true data
        # TODO: Could be pre-computed (using list0 in batch to avoid endless recomputation
        labels = []
        for i_batch in range(log_softmax.shape[1]):
            label_length = batch['label_lengths'][i_batch]
            label = batch['label'][i_batch, :label_length]
            label = [ alphabet_mapper.index_to_character(c) for c in label ]
            label = "".join(label)
            labels.append(label)

        cer = char_error_rate(preds=decoded_texts, target=labels)
        wer = word_error_rate(preds=decoded_texts, target=labels)

        df_data['name'].append(name)
        df_data['type'].append('cer')
        df_data['value'].append(cer.item())

        df_data['name'].append(name)
        df_data['type'].append('wer')
        df_data['value'].append(wer.item())

df = pd.DataFrame.from_dict(df_data)

In [31]:
df.groupby(['name', 'type']).mean()

value
name  type          
train cer   0.052477
      wer   0.235198
val   cer   0.079784
      wer   0.316895

Is that working indeed?!?! :-O Strangely, the train values change (both cer and wer) when re-executing but not the `val` values. Question: also when re-executing the cell?

### Load data: From XournalPagewise

In [ ]:
TODO: START FROM HERE NEXT TIME!

In [34]:
dm = IAMOnDBDataModule(
    '../data/datasets/2024-02-16-xournal_dataset.xoj',
    batch_size=64,
    train_val_test_split=[0.8, 0.2, 0],
    num_workers=4,
    pin_memory=True,
    # limit=200,
    limit=-1,
    transform='XournalPagewise_carbune_xyn',
)
dm.setup()
dl_val = dm.val_dataloader()
dl_train = dm.train_dataloader()

ValueError: `transform` set to non-existent value

### Do inference: XournalPagewise

In [28]:
df_data = {
    'name': [],
    'type': [],
    'value': [],
}

for name, dloader in {'train': dl_train, 'val': dl_val }.items():
    for sample_batched in dloader:
        batch = sample_batched # Just a shortcut

        log_softmax = model(sample_batched['ink'])

        decoded_texts = decoder(log_softmax, alphabet_mapper)

        # TODO: Put this in function to use both here and in model training -> or just use underlying true data
        # TODO: Could be pre-computed (using list0 in batch to avoid endless recomputation
        labels = []
        for i_batch in range(log_softmax.shape[1]):
            label_length = batch['label_lengths'][i_batch]
            label = batch['label'][i_batch, :label_length]
            label = [ alphabet_mapper.index_to_character(c) for c in label ]
            label = "".join(label)
            labels.append(label)

        cer = char_error_rate(preds=decoded_texts, target=labels)
        wer = word_error_rate(preds=decoded_texts, target=labels)

        df_data['name'].append(name)
        df_data['type'].append('cer')
        df_data['value'].append(cer.item())

        df_data['name'].append(name)
        df_data['type'].append('wer')
        df_data['value'].append(wer.item())

df = pd.DataFrame.from_dict(df_data)

In [29]:
df.groupby(['name', 'type']).mean()

value
name  type       
train cer     1.0
      wer     1.4

## Attempt n

- Check point 3 references 1-4.
- Use predict step from PL instead?